In [34]:
import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict

randomlist = []
demand = {}
random.seed(4) # Use the same random seed
for i in range(0,20):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(20):
    demand['D'+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in demand.items():
    sum = sum + v

supply = {}
for i in range(sum):
    supply[i] = 1.0

np.random.seed(4) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,20, size=(sum,20))).astype(float)

random.seed(4) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(4) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(4) # Use the same random seed
doc = {}
for i in range(0,20):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom
df_tdkom.columns = [("D"+str(i)) for i in range(1,21)]
dk = df_tdkom.transpose()
costs = dk.to_dict()

demand_s = demand.copy()
supply_s = supply.copy()
cold = sorted(demand.keys())
cols = sorted(demand.keys())

In [35]:
costs1=copy.deepcopy(costs)
costs2=copy.deepcopy(costs)
costs3=copy.deepcopy(costs)
costs4=copy.deepcopy(costs)

for i in supply:
    mi=min(costs[i].values())
    #print(costs[i])
    # print(mi)
    for j in costs2[i]:
        costs2[i][j]-=mi
# print(costs2)
for i in demand :
    mi=10000
    for j in supply:
        if costs[j][i]<mi :
            mi=costs[j][i]
    for j in supply:
        costs3[j][i]=costs3[j][i]-mi
# print(costs3)

for i in demand:
    for j in supply:
        costs[j][i]= costs2[j][i]+costs3[j][i]

costs5=copy.deepcopy(costs)
costs6=copy.deepcopy(costs)


ma=0

In [36]:
### Weighted Transportation Matrix by Demand (wcd)

res = dict((k, defaultdict(int)) for k in costs5)
rij={}
for x in supply:
    dd={}
    for y in demand:
        dd[y]=0
    rij[x]=dd

# Proportional Demand Matrix
for x in supply:
    for y in demand:
        rij[x][y]= demand[y]/supply[x]

# print(rij)
for x in supply:
    for y in demand:
        rij[x][y]*=costs5[x][y]


In [37]:
while supply or demand:
    mi=10000000000000
    l=[]
    for x in supply:
        for y in demand:
            if rij[x][y]<mi:
                mi=rij[x][y]
                l=[x,y]

    sup=l[0]
    dem=l[1]
    v=min(supply[sup],demand[dem])
    demand[dem]-=v
    supply[sup]-=v
    res[sup][dem]+=v
    if supply[sup]==0:
        del supply[sup]
    if demand[dem]==0:
        del demand[dem]


#print(demand,supply)


In [38]:
cost_d = 0
for g in sorted(costs4):
  # print (g, "\t",)
  for n in cold:
    y = res[g][n]
    if y != 0:
        pass
      # print (y,)
    cost_d += y * costs4[g][n]
    # print ("\t",)
  # print()
# print ("\n\nTotal Cost = ", cost)

In [39]:
cost_d

218113.76365580858

In [40]:
### Weighted Transportation Matrix by Supply (wcd)

res_s = dict((k, defaultdict(int)) for k in costs6)
rij_s={}
for x in supply_s:
    dd={}
    for y in demand_s:
        dd[y]=0
    rij_s[x]=dd

for x in supply_s:
    for y in demand_s:
        rij_s[x][y]= supply_s[x]/demand_s[y]
# print(rij)
for x in supply_s:
    for y in demand_s:
        rij_s[x][y]*=costs6[x][y]

In [41]:
while supply_s or demand_s:
    mi=10000000000000
    ls=[]
    for x in supply_s:
        for y in demand_s:
            if rij_s[x][y]<mi:
                mi=rij_s[x][y]
                ls=[x,y]

    sup_s=ls[0]
    dem_s=ls[1]
    v1=min(supply_s[sup_s],demand_s[dem_s])
    demand_s[dem_s]-=v1
    supply_s[sup_s]-=v1
    res_s[sup_s][dem_s]+=v1
    if supply_s[sup_s]==0:
        del supply_s[sup_s]
    if demand_s[dem_s]==0:
        del demand_s[dem_s]

#print(demand_s,supply_s)

In [42]:
cost_s = 0
for g in sorted(costs4):
  # print (g, "\t",)
  for n in cols:
    y = res_s[g][n]
    if y != 0:
        pass
      # print (y,)
    cost_s += y * costs4[g][n]
    # print ("\t",)
  # print()
# print ("\n\nTotal Cost = ", cost)

In [43]:
cost_s

211555.95188496594

In [44]:
min(cost_d,cost_s)

211555.95188496594